In [2]:
import sys
sys.path.append('..')

In [129]:
from copy import deepcopy
import random
from library.problems.tsp import TSPSolution
from library.problems.data.tsp_data import distance_matrix
from library.algorithms.genetic_algorithms.algorithm import genetic_algorithm
from library.algorithms.genetic_algorithms.mutation import swap_mutation
from library.algorithms.genetic_algorithms.selection import fitness_proportionate_selection

## Specialized Genetic Operators

In the previous notebook, we explored some traditional genetic operators: **standard crossover, binary mutation, and swap mutation**.

However, these traditional operators generate invalid solutions when applied to combinatorial problems such as the Traveling Salesman Problem (TSP), job scheduling, or vehicle routing, where solutions are represented as **permutations**.

In this notebook, we’ll explore **specialized genetic operators**, specifically designed to handle permutations without producing invalid solutions. We’ll explore one crossover and one mutation methods that respects permutation constraints.

### Why Standard Crossover/Mutation Fail for Permutation Problems

In permutation-based problems (e.g., [1, 2, 3, 4, 5]), each gene must appear exactly once.
Standard genetic operators like one-point crossover or value-flip mutation can break this rule, resulting in invalid offspring with duplicates or missing values.


#### Standard Mutation - Value Flip

This mutation is inspired by the standard binary mutation: each gene is randomly replaced with another value with some probability.

Individual [1, 2, 3, 4, 5]

Mutated individual (hypothetical): [1, 2, 4, 1, 5] ❌ (Duplicate '1')

#### Standard Crossover

Parent 1: [1, 2 | 3, 4, 5]

Parent 2: [3, 4 | 5, 1, 2]


Offspring 1 (invalid): [1, 2, 5, 1, 2] ❌ (Duplicates '1' and '2')

Offspring 2 (invalid): [3, 4, 3, 4, 5] ❌ (Duplicates '3' and '4')

### Cycle Crossover

Cycle Crossover keeps items in their original positions across parents by identifying cycles of indices where elements should remain fixed.

**Pseudo-code:**

1. Choose random index in Parent 1 and copy the element to first child.
3. Copy element in same index in Parent 2 to second child.
4. Find this element in Parent 1 and copy it to first child, and repeat the process.
5. Once the cycle completes (we end up back on the initial index), the remaining positions are filled from the other parent.

![Cycle Crossover](images/cycle-xo.png)


In [ ]:
def cycle_crossover(parent1_repr: str | list, parent2_repr: str | list):
    """
    Performs Cycle Crossover (CX) between two parents

    Cycle Crossover preserves the position of elements by identifying a cycle
    of indices where the values from each parent will be inherited by each offspring.
    The remaining indices are filled with values from the other parent, maintaining valid permutations.

    Args:
        parent1_repr (str or list): The first parent representation.
        parent2_repr (str or list): The second parent representation.
            Both parents must have the same length and type.

    Returns:
        tuple: Two offspring permutations resulting from the crossover.
    """
    # Randomly choose a starting index for the cycle
    initial_random_idx = random.randint(0, len(parent1_repr)-1)

    # Initialize the cycle with the starting index
    cycle_idxs = [initial_random_idx]
    current_cycle_idx = initial_random_idx

    # Traverse the cycle by following the mapping from parent2 to parent1
    while True:
        value_parent2 = parent2_repr[current_cycle_idx]
        # Find where this value is in parent1 to get the next index in the cycle
        next_cycle_idx = parent1_repr.index(value_parent2)

        # Closed the cycle -> Break
        if next_cycle_idx == initial_random_idx:
            break

        cycle_idxs.append(next_cycle_idx)
        current_cycle_idx = next_cycle_idx
    
    offspring1_repr = []
    offspring2_repr = []
    for idx in range(len(parent1_repr)):
        if idx in cycle_idxs:
            # Keep values from parent1 in offspring1 in the cycle indexes
            offspring1_repr.append(parent1_repr[idx])
            # Keep values from parent2 in offspring2 in the cycle indexes
            offspring2_repr.append(parent2_repr[idx])
        else:
            # Swap elements from parents in non-cycle indexes
            offspring1_repr.append(parent2_repr[idx])
            offspring2_repr.append(parent1_repr[idx])

    # To keep the same type as the parents representation
    if isinstance(parent1_repr, str) and isinstance(parent2_repr, str):
        offspring1_repr = "".join(offspring1_repr)
        offspring2_repr = "".join(offspring2_repr)

    return offspring1_repr, offspring2_repr

Let's test it.

In [61]:
parent1 = [1, 2, 3, 4, 5, 6, 7]
parent2 = [3, 4, 2, 1, 7, 5, 6]

print("Parent 1:", parent1)
print("Parent 2:", parent2)

offspring1_repr, offspring2_repr = cycle_crossover(parent1_repr=parent1, parent2_repr=parent2)

print("Offspring 1:", offspring1_repr)
print("Offspring 2:", offspring2_repr)

Parent 1: [1, 2, 3, 4, 5, 6, 7]
Parent 2: [3, 4, 2, 1, 7, 5, 6]
Offspring 1: [1, 2, 3, 4, 7, 5, 6]
Offspring 2: [3, 4, 2, 1, 5, 6, 7]


### Inversion Mutation

Inversion mutation works by selecting two random indices and reversing the subsequence between them, with a certain probability.

![Inversion Mutation](images/inversion-mutation.png)

In [ ]:
def inversion_mutation(representation: str | list, mut_prob):
    """
    Applies inversion mutation to a representation.

    Inversion mutation selects two random indices and reverses the 
    subsequence between them, with a certain probability.

    Parameters:
    ----------
    representation : str or list
        The individual to mutate. Should represent a valid permutation.
    mut_prob : float
        Probability of applying the mutation (between 0 and 1).

    Returns:
    -------
    str or list
        A new individual with the mutated representation (if mutation occurs),
        or a copy of the original.
    """
    if random.random() <= mut_prob:
        # Select two distinct indices
        first_idx = random.randint(0, len(representation)-1)
        second_idx = first_idx
        # We want to get two indexes that are at least 2 genes away
        while abs(second_idx-first_idx) <= 1:
            second_idx = random.randint(0, len(representation)-1)
    
        # Ensure first_idx < second_idx
        if first_idx > second_idx:
            first_idx, second_idx = second_idx, first_idx

        # Reverse between first and second index
        reversed_subsequence = list(reversed(representation[first_idx:second_idx]))

        # Convert back to string if original representation is a string
        if isinstance(representation, str):
            reversed_subsequence = "".join(reversed_subsequence)

        # Keep everything from second index (excluding it) until the end
        new_representation = representation[:first_idx] + reversed_subsequence + representation[second_idx:]
        return new_representation
    else:
        return deepcopy(representation)

Let's test it.

In [125]:
representation = [1, 2, 3, 4, 5, 6, 7]

inversion_mutation(representation, mut_prob=1)

[1, 2, 7, 6, 5, 4, 3]

## Solving TSP with Genetic Algorithms

Just like we did in the previous notebook for the Knapsack Problem, we’ll now solve TSP using genetic algorithms.

To structure our solution, we’ll define a `TSPGASolution` class where we’ll define in the crossover and mutation methods needed to run the `genetic_algorithm` function.

To keep the class flexible and reusable, we’ll pass the `mutation_function` and `crossover_function` as callable arguments when creating an instance of `TSPGASolution`.

### A small but important side note

TSP solutions are represented as permutations of city indices, where the path must start and end at the same city (i.e., the starting index is fixed at both ends).

When applying permutation-based operators like cycle crossover or inversion mutation, we need to preserve this constraint. That means we should only apply genetic operators to the middle portion of the route: excluding the first and last cities, which must remain the same.

So in practice, our crossover and mutation functions will only operate on the inner part of the individual, keeping the boundaries intact.

In [ ]:
class TSPGASolution(TSPSolution):
    def __init__(
        self,
        distance_matrix,
        starting_idx,
        mutation_function, # Callable
        crossover_function, # Callable
        repr = None,
    ):
        self.mutation_function = mutation_function
        self.crossover_function = crossover_function

        super().__init__(
            distance_matrix=distance_matrix,
            starting_idx=starting_idx,
            repr=repr,
        )
    
    def mutation(self, mut_prob):
        """
        Applies the provided mutation operator to the middle portion 
        of the route (excluding start and end cities).
        """
        # Apply mutation to the middle route segment
        middle_segment = self.repr[1:-1]  # Exclude starting/ending city
        mutated_segment = self.mutation_operator(middle_segment, mut_prob)
        new_repr = [self.starting_idx] + mutated_segment + [self.starting_idx]
        
        return TSPGASolution(
            distance_matrix=self.distance_matrix,
            starting_idx=self.starting_idx,
            mutation_function=self.mutation_function,
            crossover_function=self.crossover_function,
            repr=new_repr
        )

    def crossover(self, other_solution):
        """
        Applies the provided crossover operator to the middle portions
        of two parent routes (excluding start/end cities), and returns
        two new offspring solutions.
        """
        # Apply crossover to the middle route segment of the parents
        parent1_middle = self.repr[1:-1]
        parent2_middle = other_solution.repr[1:-1]

        offspring1_middle, offspring2_middle = self.crossover_function(parent1_middle, parent2_middle)

        offspring1_repr = [self.starting_idx] + offspring1_middle + [self.starting_idx]
        offspring2_repr = [self.starting_idx] + offspring2_middle + [self.starting_idx]

        return (
            TSPGASolution(
                distance_matrix=self.distance_matrix,
                starting_idx=self.starting_idx,
                mutation_function=self.mutation_function,
                crossover_function=self.crossover_function,
                repr=offspring1_repr
            ),
            TSPGASolution(
                distance_matrix=self.distance_matrix,
                starting_idx=self.starting_idx,
                mutation_function=self.mutation_function,
                crossover_function=self.crossover_function,
                repr=offspring2_repr
            )
        )

Let's apply the genetic algorithm to TSP using cycle crossover and inversion mutation. Here the probability of mutation should be relatively small because inversion mutation may be very destructive.

In [ ]:
POP_SIZE = 50
initial_population = [
    TSPGASolution(
        distance_matrix=distance_matrix,
        starting_idx=0,
        crossover_function=cycle_crossover,
        mutation_function=inversion_mutation
    )
    for _ in range(POP_SIZE)
]

best_solution = genetic_algorithm(
    initial_population=initial_population,
    max_gen=100,
    selection_algorithm=fitness_proportionate_selection,
    maximization=False,
    xo_prob=0.8,
    mut_prob=0.2,
    elitism=True,
    verbose=True
)

print("Best solution fitness", best_solution.fitness())

And now let's use cycle crossover again, but this time we use swap mutation.

Now we can set a higher probability of miutation because swap mutation is less destructive than inversion mutation.

In [ ]:
initial_population = [
    TSPGASolution(
        distance_matrix=distance_matrix,
        starting_idx=0,
        crossover_function=cycle_crossover,
        mutation_function=swap_mutation
    )
    for _ in range(POP_SIZE)
]

best_solution = genetic_algorithm(
    initial_population=initial_population,
    max_gen=100,
    selection_algorithm=fitness_proportionate_selection,
    maximization=False,
    xo_prob=0.8,
    mut_prob=0.2,
    elitism=True,
    verbose=True
)

print("Best solution fitness", best_solution.fitness())